# Recommendation with graph
Adapted from: http://tinkerpop.apache.org/docs/current/recipes/#recommendation

### Tinkerpop Documentation
In Python, as, in, and, or, is, not, from, and global are reserved words
http://tinkerpop.apache.org/docs/current/reference/#gremlin-python

In [ ]:
!pip install gremlinpython

In [ ]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection


from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier

In [ ]:
graph = Graph()

driver = DriverRemoteConnection('ws://<PUT_YOUR_NEPTUNE_URL_HERE>:8182/gremlin','g')
g = graph.traversal().withRemote(driver)

## Clean everything

In [ ]:
g.V().drop().iterate().toList()

In [ ]:
g.V().properties().toList()

In [ ]:
g.E().toList()

## Add vertices

In [ ]:
for n in ["alice", "bob", "jon", "jack", "jill"]:
    g.addV("person").property("name",n ).iterate()

In [ ]:
for i in range(1, 11):
    g.addV("product").property("name","product #%d" % i).iterate()

## Add edges

In [ ]:
for i in range(3, 8):
    product = g.V().has("product", "name","product #%d" % i)
    g.V().has("person", "name", "alice").addE('bought').to(product).iterate()

In [ ]:
for i in range(1, 6):
    g.V().has("person", "name", "bob").addE('bought').to(g.V().has("product", "name","product #%d" % i)).iterate()

In [ ]:
for i in range(6, 11):
    g.V().has("person", "name", "jon").addE('bought').to(g.V().has("product", "name","product #%d" % i)).iterate()

In [ ]:
for i in range(11):
    i += 1
    if i%2==0:
        g.V().has("person", "name", "jack").addE('bought').to(g.V().has("product", "name","product #%d" % i)).iterate()

In [ ]:
for i in range(11):
    if i%2==1:
        g.V().has("person", "name", "jill").addE('bought').to(g.V().has("product", "name","product #%d" % i)).iterate()

## Some queries

In [ ]:
statics.load_statics(globals())

#### The first step to making a recommendation to "alice" using collaborative filtering is to understand what she bought:

In [ ]:
g.V().has('name','alice').out('bought').properties().toList()

![RecommendationAlice1](http://tinkerpop.apache.org/docs/current/images/recommendation-alice-1.png)
> The next step is to determine who else purchased those products:

In [ ]:
g.V().has('name','alice').out('bought').in_('bought').dedup().values('name').toList()

> It is worth noting that "alice" is in the results above. She should really be excluded from the list as the interest is in what individuals other than herself purchased:

In [ ]:
g.V().has('name','alice').as_('her').out('bought').in_('bought').where(P.neq('her')).dedup().values('name').toList()

> The following diagram shows "alice" and those others who purchased "product #5".
![RecommendationAlice2](http://tinkerpop.apache.org/docs/current/images/recommendation-alice-2.png)
> The knowledge of the people who bought the same things as "alice" can then be used to find the set of products that they bought:

In [ ]:
g.V().has('name','alice').as_('her').out('bought').in_('bought').where(P.neq('her')).out('bought').dedup().values('name').toList()

![RecommendationAlice3](http://tinkerpop.apache.org/docs/current/images/recommendation-alice-3.png)
> This set of products could be the basis for recommendation, but it is important to remember that "alice" may have already purchased some of these products and it would be better to not pester her with recommendations for products that she already owns. Those products she already purchased can be excluded as follows:



In [ ]:
g.V().has('name','alice').as_('her').out('bought').aggregate('self').in_('bought').where(P.neq('her')).out('bought').where(P.without('self')).dedup().values('name').toList()

![RecommendationAlice4](http://tinkerpop.apache.org/docs/current/images/recommendation-alice-4.png)
> The final step would be to group the remaining products (instead of dedup() which was mostly done for demonstration purposes) to form a ranking:

In [ ]:
g.V().has('person','name','alice').as_('her').out('bought').aggregate('self').in_('bought').where(P.neq('her')).out('bought').where(P.without('self')).groupCount().order().toList()



- Find "alice" who is the person for whom the product recommendation is being made.

- Traverse to the products that "alice" bought and gather them for later use in the traversal.

- Traverse to the "person" vertices who bought the products that "alice" bought and exclude "alice" herself from that list.

- Given those people who bought similar products to "alice", find the products that they bought and exclude those that she already bought.

- Group the products and count the number of times they were purchased by others to come up with a ranking of products to recommend to "alice".

> The previous example was already described as "basic" and obviously could take into account whatever data is available to further improve the quality of the recommendation (e.g. product ratings, times of purchase, etc.). One option to improve the quality of what is recommended (without expanding the previous dataset) might be to choose the person vertices that make up the recommendation to "alice" who have the largest common set of purchases.

> Looking back to the previous code example, consider its more strip down representation that shows those individuals who have at least one product in common:

In [ ]:
g.V().has('person','name','alice').as_('her').out('bought').aggregate('self').in_('bought').where(
    P.neq('her')
).out('bought').where(
    P.without('self')
).groupCount().order(local).by(values, Order.decr).toList()                                                   

> Next, do some grouping to find count how many products they have in common:

In [ ]:
g.V().has("person","name","alice").as_("alice").out("bought").aggregate("self").in_("bought").where(
    P.neq("alice")
).dedup().group().by().by(out("bought").where(P.within("self")).count()).toList()

> Now that there is a list of "person" vertices to base the recommendation on, traverse to the products that they purchased:

In [ ]:
g.V().has("person","name","alice").as_("alice").out("bought").aggregate("self").in_("bought").where(
    neq("alice")
).dedup().group().by().by(
    out("bought").where(within("self")).count()
).as_("g").select(values).order(local).by(Order.decr).limit(local, 1).as_("m").select("g").unfold().where(
    select(values).as_("m")
).select(keys).out("bought").where(without("self")).toList()

> The above output shows that one product is held in common making it the top recommendation:

In [ ]:
g.V().has("person","name","alice").as_("alice").out("bought").aggregate("self").in_("bought").where(
   neq("alice")
).dedup().group().by().by(out("bought").where(
    within("self")
).count()).as_("g").select(values).order(local).by(decr).limit(local, 1).as_("m").select("g").unfold().where(
    select(values).as_("m")
).select(keys).out("bought").where(
    without("self")
).groupCount().order(local).by(values, Order.decr).by(
    select(keys).values("name")
).unfold().select(keys).values("name").toList()

> In considering the practical applications of this recipe, it is worth revisiting the earlier "basic" version of the recommendation algorithm:

In [ ]:
g.V().has('person','name','alice').as_('her').out('bought').aggregate('self').in_('bought').where(
    neq('her')
).out('bought').where(
    without('self')
).groupCount().order(local).by(values, Order.decr).toList()

> The above traversal performs a full ranking of items based on all the connected data. That could be a time consuming operation depending on the number of paths being traversed. As it turns out, recommendations don’t need to have perfect knowledge of all data to provide a "pretty good" approximation of a recommendation. It can therefore make sense to place additional limits on the traversal to have it better return more quickly at the expense of examining less data.

> Gremlin provides a number of steps that can help with these limits like: coin(), sample(), and timeLimit(). For example, to have the traversal sample the data for no longer than one second, the previous "basic" recommendation could be changed to:

In [ ]:
g.V().has('person','name','alice').as_('her').out('bought').aggregate('self').in_('bought').where(
    neq('her')
).out('bought').where(
    without('self')
).timeLimit(1000).groupCount().order(local).by(values, Order.decr).toList()

> In using sampling methods, it is important to consider that the natural ordering of edges in the graph may not produce an ideal sample for the recommendation. For example, if the edges end up being returned oldest first, then the recommendation will be based on the oldest data, which would not be ideal. As with any traversal, it is important to understand the nature of the graph being traversed and the behavior of the underlying graph database to properly achieve the desired outcome.

# Well Done!